<a href="https://colab.research.google.com/github/olaviinha/NeuralImageSuperResolution/blob/master/SuperRes_ESRGAN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Image Super-Resolution

Colab for [JoeyBallentine's fork](https://github.com/JoeyBallentine/ESRGAN) of [BlueAmulet's fork](https://github.com/BlueAmulet/ESRGAN) of [ESRGAN by Xinntao](https://github.com/xinntao/ESRGAN), implementation of [Enhanced Super-Resolution Generative Adversarial Networks](https://arxiv.org/abs/1809.00219).

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisities.

import os
force_setup = False

pip_packages = 'typer rich gmic'

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

%cd /content/
!git clone https://github.com/olaviinha/ESRGAN.git Colab-ESRGAN
!git clone https://github.com/olaviinha/BlurDetection2.git
%cd "/content/Colab-ESRGAN"

dir_tmp = '/content/tmp/'
dir_mask = '/content/tmp/mask/'
dir_input = '/content/Colab-ESRGAN/input/'
dir_dejpeg = '/content/Colab-ESRGAN/dejpeg/'
dir_upscaled = '/content/Colab-ESRGAN/upscaled/'
dir_output = '/content/Colab-ESRGAN/output/'
dir_models = '/content/Colab-ESRGAN/models/'
create_dirs([dir_tmp, dir_input, dir_dejpeg, dir_upscaled, dir_output, dir_models])

#----------------------------------------

import requests, gmic, cv2
from google.colab import output
from google.colab.patches import cv2_imshow
from IPython.display import Image
from datetime import datetime
force_setup = False

#----------------------------------------

# RRDB ESRGAN & RRDB PSNR by Xintao Wang, Ke Yu, Shixiang Wu, Jinjin Gu, Yihao Liu, Chao Dong, Yu Qiao, Chen Change Loy
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-' -O models/RRDB_ESRGAN_x4.pth
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr' -O models/RRDB_PSNR_x4.pth
# fatal_anime by twittman
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/x99pKzS7TNaErrC/download" -O models/4x_FatalAnime_500000_G.pth
# fatal_pixels twittman
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/mDGmi7NgdyyQRXL/download?path=%2F&files=4x_FatalPixels_340000_G.pth&downloadStartSecret=r4q3aw60ijm" -O models/4x_FatalPixels.pth
# deJpeg_Fatality_PlusULTRA by twittman
!wget --no-check-certificate "https://de-next.owncube.com/index.php/s/w82HLrLWmWi4SQ5/download" -O models/1x_DeJpeg_Fatality_PlusULTRA.pth
# 4x_NMKD-YandereNeo-Lite_320k by nmkd
!gdown --id 14lA-Ks5quxheNyVeXRvzeoSAOm6ISDHn -O "/content/Colab-ESRGAN/models/4x_NMKD-YandereNeo-Lite_320k.pth"

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@title # Upscale
#@markdown <small>Please provide `input_image` and `output_dir` relative to your Google Drive root.</small>
input_image = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}
# All models:
model = "ESRGAN_and_FatalPixels" #@param ["ESRGAN_and_FatalPixels", "ESRGAN_and_PSNR", "ESRGAN", "PSNR", "FatalAnime", "FatalPixels", "NMKD"]
# Selected models:
# model = "PSNR" #@param ["ESRGAN_and_PSNR", "ESRGAN", "PSNR", "FatalAnime", "FatalPixels", "deJpeg", "NMKD"]
## #@markdown <small>If you selected ESRGAN_and_PSNR, select interpolation ratio. 0 = ESRGAN only, 100 = PSNR only</small>
## ESRGAN_PSNR_ratio = 0 #@param {type:"slider", min:0, max:100, step:1}
#@markdown <small>When two models are selected from model menu, select ratio in which they are used. 0 = first model, 100 = second model.</small>
interpolation = 90 #@param {type:"slider", min:0, max:100, step:1}
dejpeg = True #@param {type:"boolean"}
sharpen = False #@param {type:"boolean"}



#----------------------------------------------------------
# #@markdown <hr color="#666">

# #@markdown <small>Timestamp output files (YYYYMMDDHHIISS).</small>
# timestamp = False #@param {type:"boolean"}
# #@markdown ### Dev options
# blur_mask = False #@param {type:"boolean"}
# blur_threshold = 39 #@param {type:"slider", min:0, max:200, step:1}
# show_progress = True #@param {type:"boolean"}
# show_result = True #@param {type:"boolean"}
#----------------------------------------------------------

sharpen_before = True
sharpen_after = False

show_state = show_progress

if os.listdir(dir_input):
  clean_dirs([dir_input, dir_output])

input_image = drive_root+input_image
if output_dir == '' or output_dir == 'same':
  output_dir = path_dir(input_image)
else:
  output_dir = fix_path(drive_root+output_dir)

if timestamp is True:
  uniq_id = datetime.today().strftime('%Y%m%d%H%M%S')
else:
  uniq_id = requests.get('https://api.inha.asia/k/?type=short').text

%cd "/content/Colab-ESRGAN"

model_1_part = interpolation
model_2_part = 100-interpolation

if model == 'ESRGAN_and_FatalPixels':
  # 25 75
  model_file = 'RRDB_ESRGAN_x4.pth@'+str(model_1_part)+'|4x_FatalPixels.pth@'+str(model_2_part)
if model == 'ESRGAN_and_PSNR':
  # 70 30
  model_file = 'RRDB_ESRGAN_x4.pth@'+str(model_1_part)+'|RRDB_PSNR_x4.pth@'+str(model_2_part)
if model == 'ESRGAN':
  model_file = 'RRDB_ESRGAN_x4.pth'
if model == 'PSNR':
  model_file = 'RRDB_PSNR_x4.pth'
if model == 'FatalAnime':
  model_file = '4x_FatalAnime_500000_G.pth'
if model == 'FatalPixels':
  model_file = '4x_FatalPixels.pth'
if model == 'DeJpeg':
  model_file = '1x_DeJpeg_Fatality_PlusULTRA.pth'
if model == 'NMKD':
  model_file = 'NMKD-YandereNeo_lite_320k_x4.pth'

# input_file = dir_output+basename(input_image)+'.png'
# tmp_file = output_dir+basename(input_image)+'_'+uniq_id+'_'+model+'_'+str(model_1_part)+'_'+str(model_2_part)+path_ext(input_image)
# output_file = tmp_file
# state = input_file

final_file = output_dir+basename(input_image)+'_'+uniq_id+'_'+model+'_'+str(model_1_part)+'_'+str(model_2_part)+path_ext(input_image)
state = input_image
!cp "{input_image}" "{dir_input}"

if dejpeg:
  op(c.title, 'DeJpeg...')
  !python upscale.py 1x_DeJpeg_Fatality_PlusULTRA.pth --input "{dir_input}" --output "{dir_dejpeg}"
  state = dir_dejpeg+os.listdir(dir_dejpeg)[0]
  if show_state: Image(state)
  op(c.ok, 'Done.\n')


## Sharpen
if sharpen and shapen_before:
  op(c.title, 'Sharpen...')
  tmp_file = dir_tmp+basename(input_image)+'_sharpened.png'
  gmic.run("input "+state+" fx_mighty_details 35,1,2,1,11,0 output "+tmp_file)
  # !cp "{tmp_file}" "{dir_input}"
  # state = input_file
  # state = dir_input+os.listdir(dir_input)[0]
  state = tmp_file
  if show_state: Image(state)
  op(c.ok, 'Done.\n')

!cp "{state}" "{dir_input}"

## Neural upscale
op(c.title, 'Upscale...')
!python upscale.py "{model_file}" --input "{dir_input}" --output "{dir_upscaled}"
# state = state.replace('input/', 'output/')
state = dir_upscaled+os.listdir(dir_upscaled)[0]
if show_state: Image(state)
op(c.ok, 'Done.\n')

## Sharpen
if sharpen and shapen_after:
  op(c.title, 'Sharpen...')
  tmp_file = dir_tmp+basename(input_image)+'_sharpened.png'
  gmic.run("input "+state+" fx_mighty_details 35,1,2,1,11,0 output "+tmp_file)
  state = tmp_file
  if show_state: Image(state)
  op(c.ok, 'Done.\n')

## Blur map
##
## To handle blurred areas differently. Work in progress.
##
if blur_mask:
  # blur_input_img = state
  blur_input_img = input_image
  blurmapSS = dir_tmp+'blur_mask_ss'
  blurmapSL = dir_tmp+'blur_mask_sl'
  blurmapBL = dir_tmp+'blur_mask_bl.png'
  %cd /content/BlurDetection2

  op(c.title, 'Generate blur map...')
  !python process.py -i "{blur_input_img}" -s {blurmapSS} -t "{blur_threshold}" -f
  state = blurmapSS+'_blur_map.png'
  if show_state: Image(state)
  op(c.ok, 'Done.\n')

  input_size = cv2.imread(state)
  width = int(input_size.shape[1])
  height = int(input_size.shape[0])
  dim = (width, height)
  temp_image = cv2.imread(state)
  upscaled = cv2.resize(temp_image, dim, interpolation = cv2.INTER_AREA)

  state = blurmapSL+'_blur_map.png'
  if show_state: Image(state)
  cv2.imwrite(state, upscaled)
  ba = str(width/200)
  gmic.run("-input "+blurmapSL+'_blur_map.png'+" -blur_linear "+ba+","+ba+",0 -glow 5 -adjust_colors 0,0,10,0,0 -adjust_colors 10,50,0,0,0 -output "+blurmapBL)
  if show_state: Image(blurmapBL)
  op(c.ok, 'Done.\n')

  !cp /content/tmp/blur_mask_bl.png /content/drive/MyDrive/ai/st3/test/
  !cp /content/tmp/blur_mask_sl_blur_map.png /content/drive/MyDrive/ai/st3/test/
  !cp /content/tmp/blur_mask_ss_blur_map.png /content/drive/MyDrive/ai/st3/test/
  !cp /content/tmp/blur_mask_ss_pretty_blur_map.png /content/drive/MyDrive/ai/st3/test/

!cp "{state}" "{final_file}"
if os.path.isfile(final_file):
  op(c.ok, '\nFIN.', 'File saved as '+final_file.replace(drive_root, ''))
if show_result: Image(final_file)